In [1]:
import time
import serial
import csv
import numpy as np
import matplotlib.pylab as plt
from matplotlib import colors
from IPython import display
def sendcmd(command, resp=True):
    ser = serial.Serial(
        port='COM6', #Change to port of temperature controller on device
        #For Model 336 Lake Shore. Check manual for your controller.
        baudrate=57600, 
        parity=serial.PARITY_ODD,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.SEVENBITS
    )
    if ser.isOpen() == 0:
        ser.open()
    comando = command + '\r\n'
    #print('comando: ' + comando)
    ser.write(comando.encode())  # Encode the command as bytes before writing
    out = b''  # Initialize out as bytes
    time.sleep(0.5)  # Wait one second before reading output
    while ser.inWaiting() > 0:
        out += ser.read(1)
    ser.close()
    return out.decode()  # Decode bytes into a string before returning

#timeLimit = float(input("Experiment time limit in seconds (float)"))
timeLimit = 60*60
#timeHeaterOff = float(input("Time to wait before starting in seconds (float)"))
timeHeaterOff = 60*5
#percent = float(input("Output percentage (float 0-100)"))
percent = 57
#run = float(input("Run number for this percentage (int)"))
run = 1
timeStamp = []; T_sensor = []; output = []

t_start = time.time()
t_end = time.time() + timeLimit
#sleep_max = 1.0
stepStarted = False #Whether step test has begun
prevTime = 0

#fig, ax = plt.subplots()
#fig.set_size_inches(7, 5)
#ax.set_xlim(0, timeLimit) 
#ax.set_ylim(103.0, 103.1) #changed from 1000
#ax.set_xlabel('time (seconds)', size=12)
#ax.set_ylabel('temperature (Kelvin)', size=12)

filename = 'Results/steptest' + str(percent) + 'percent' + str(run) + '.csv'
fileheaders = ['time', 'T_sensor', 'Output']

with open(filename, 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile) 
    writer.writerow(fileheaders) 

initialOutput = sendcmd('MOUT? 1')
while time.time() < t_end:
    #if len(timeStamp) != 0:
    #    prev_time = timeStamp[-1]
    timeStamp.append(time.time() - t_start)
    T_sensor.append(float(sendcmd('KRDG? A')))
    if stepStarted == True:
        output.append(percent)
    else:
        output.append(initialOutput)
        
    if ((time.time() - t_start > timeHeaterOff) and (stepStarted == False)):
        sendcmd('MOUT 1,' + str(percent))
        testTime = timeStamp[-1]
        stepStarted = True
        print("Output changed at ", testTime, ", index ", len(timeStamp)-1)
    print(round(timeStamp[-1],2), round(T_sensor[-1],3), end='\r')
    
    #Attempt at live plotting data (plot disappears after each point)
    #if timeStamp[-1] >= prevTime + 60:
    #    ax.scatter(timeStamp[-1], T_sensor[-1], color='red', 
    #               linewidth=1, marker="v", s= 40, 
    #               edgecolors='black')
    #    display.display(plt.gcf())
    #    display.clear_output(wait=True)
    #    prevTime = timeStamp[-1]
    
    #Write data to file
    with open(filename, 'a', newline='') as csvfile:  
        writer = csv.writer(csvfile) 
        writer.writerow([timeStamp[-1], T_sensor[-1], output[-1]])
        
    #Attempt to keep time stamps uniformly seperated:
    #if len(timeStamp) >= 2:
        #print("len(timeStamp) >= 2")
        #sleep = sleep_max - (timeStamp[-1] - timeStamp[-2])
    #else:
        #print("else")
        #sleep = sleep_max
    #if sleep > 0:
        #print("sleep > 0")
        #time.sleep(sleep)    
    time.sleep(1)

#Turn off heater when done
sendcmd('MOUT 1, 0.0')
print("done")

Output changed at  300.0903649330139 , index  196
done.66 147.183


In [ ]:
#curve fitting and parameter finding
for i in range(0, len(time)-1):
    if time[i] >= 60 * 5:
        startFit = i
        break
tempStart = sum(temp[startFit-50:startFit])/50
tempEnd = sum(temp[-50:-1])/50
tempTao = 0.632 * (tempEnd - tempStart) + tempStart
K_P = (tempEnd - tempStart)/percent
for i in range(0, len(time[startFit:])-1):
    if temp[i] >= tempTao:
        tao = time[i]
        break

numpy.polyfit(numpy.log(time[startFit:]), temp[startFit:], 1)

In [8]:
#get data from step test
T_min = np.min(T_sensor)
T_max = np.max(T_sensor)
T_diff = T_max - T_min
K_prop = T_diff/percent
T_rise = 0.632 * T_diff + T_min
deadtime = 0 #time bettween output and responce
tao = 0 #time to reach 63.2% of final temp
for i in range(0, len(timeStamp)-1):
    #if temp is sufficiently changed
    if ((T_sensor[i] >= 0.0001 * T_diff + T_min) and (deadtime == 0)):
        deadtime = timeStamp[i] - 60
    #if temp is >= 63.2% of the way to final temp
    if T_sensor[i] >= T_rise:
        tau = timeStamp[i] - deadtime
        break
print('deadtime is : ', deadtime, ' tao is: ', tao)

deadtime is :  -60.0  tao is:  0


In [10]:
import time
import serial
import csv
def sendcmd(command, resp=True):
    ser = serial.Serial(
        port='COM6',
        baudrate=57600,
        parity=serial.PARITY_ODD,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.SEVENBITS
    )

    if ser.isOpen() == 0:
        print('ndoikoi la nde rekaka\n')
        ser.open()
    comando = command + '\r\n'
    print('comando: ' + comando)
    ser.write(comando.encode())  # Encode the command as bytes before writing
    out = b''  # Initialize out as bytes
    time.sleep(0.5)  # Wait one second before reading output
    while ser.inWaiting() > 0:
        out += ser.read(1)
    ser.close()
sendcmd('MOUT 1, 0.0')

comando: MOUT 1, 0.0

